In [1]:
import biosteam as bst
import thermosteam as tmo
import biorefineries as bf
from biorefineries import cellulosic
# from lignin_saf.rcf import * 
from lignin_saf.reactor import *

In [2]:


def create_chemicals():


    # copying cellulosic ethanol chemicals 
    chems = cellulosic.create_cellulosic_ethanol_chemicals().copy()

    # Adding pre-existing chemicals
    methanol = tmo.Chemical('Methanol')
    hydrogen = tmo.Chemical('Hydrogen')

    
    # Custom chemical properties estimated using NIST ThermoDataEngine (TDE) from Aspen Plus V14
    # Only propylguaiacol was native to Aspen, every other component was user defined
   
    propylguaiacol = tmo.Chemical(  # S-Lignin Monomer 1
        'Propylguaiacol',
        default = True,
        search_db=False,
        formula='C10H14O2',
        phase='l',
        omega = 0.6411,
        Tb = 541.7,
        Tc = 749,
        Pc = 2.9e6,
        Hvap = 7.78e4,
        rho = 1056.3,

    )
    propylguaiacol.synonyms = ('4-Propylguaiacol',)

    
    propylsyringol = tmo.Chemical(   # G-Lignin Monomer 1
        'Propylsyringol',
        default = True,
        search_db=False,
        formula='C11H16O3',
        phase='l',
        omega = 0.87334, # accentric factor
        Tb = 617.3,      # [K]  normal boiling point
        Tc = 819,        # [K]  critical temperature
        Pc = 2565348.3,   # [Pa] critical pressure
        Hvap = 1.07e5,    # [J/mol] enthalpy of vaporization at 298 K
        rho  = 1274.3     # [kg/m3]
    )
    propylsyringol.synonyms = ('4-Propylsyringol',)

    syringaresinol = tmo.Chemical(  # S-Lignin Dimer 
        'Syringaresinol',
        default = True,
        search_db = False,
        formula = 'C22H26O8',
        phase = 'l',
        omega = 1.4608,
        Tb = 796.7,
        Tc = 979,
        Pc = 1.83e6,
        Hvap = 1.99e5,
        rho = 1596.5
    )

    g_dimer = tmo.Chemical(   # Couldn't find a chemical name so going with g_dimer
        'G_Dimer',
        default = True,
        search_db = False,
        MW = 362.42,
        phase = 'l',
        omega = 1.3966,
        Tb = 812.6,
        Tc = 992,
        Pc = 1.54e6,
        Hvap = 1.97e5,
        rho = 1497.3
    )

    s_oligomer = tmo.Chemical(
        'S_Oligomer',
        default = True,
        search_db = False,
        MW = 628.67,
        phase = 'l',
        omega = 0.78742,
        Tb = 921.4,
        Tc = 1128,
        Pc = 8.27e5
    )  

    g_oligomer = tmo.Chemical(   
        'G_Oligomer',
        default = True,
        search_db = False,
        MW = 540.65,
        phase = 'l'
    )


    

    # 4) Extend the base collection
    chems.extend([methanol, hydrogen, propylguaiacol, propylsyringol,syringaresinol,g_dimer,
                  s_oligomer, g_oligomer])

    chems.compile()
    return chems

# Usage (keep side effects explicit)
chems = create_chemicals()
bst.settings.set_thermo(chems)


In [3]:
bst.settings.CEPCI = 541.7   # CEPCI for 2016

In [4]:
chems['Extract']

Chemical: Extract (phase_ref='l') at phase='l'
[Names]  CAS: Extract
         InChI: None
         InChI_key: None
         common_name: None
         iupac_name: ()
         pubchemid: None
         smiles: None
         formula: C6H12O6
[Groups] Dortmund: <1CH2, 4CH, 1OH(P...
         UNIFAC: <1CH2, 4CH, 5OH, 1C...
         PSRK: <1CH2, 4CH, 5OH, 1CHO...
         NIST: <Empty>
[Data]   MW: 180.16 g/mol
         Tm: 419.15 K
         Tb: 844.68 K
         Tt: 419.15 K
         Tc: 755 K
         Pt: 0.21809 Pa
         Pc: 4.82e+06 Pa
         Vc: 0.000414 m^3/mol
         Hf: -1.2512e+06 J/mol
         S0: 0 J/K/mol
         LHV: 2.5606e+06 J/mol
         HHV: 2.8246e+06 J/mol
         Hfus: 19933 J/mol
         Sfus: 0
         omega: 2.387
         dipole: None
         similarity_variable: 0.13322
         iscyclic_aliphatic: 0
         combustion: {'CO2': 6, 'O2'...


In [5]:
# Defining poplar group based off composition given in Bartling et al [1] Table S1

chems.define_group(
    name='Poplar',
    IDs=['Cellulose',   # Cellulase break this down to glucose
         'Xylan',       # Hemicellulose
         'Arabinan',    # Hemicellulose
         'Mannan',      # Hemicellulose
         'Galactan',    # Hemicellulose
         'Sucrose',   
         'Lignin',
         'Acetate',
         'Extract',
         'Ash'],
    composition=[0.464,      # Dry wt composition, feed also has 20 wt% moisture content
                 0.134,
                 0.002,
                 0.037,
                 0.014,
                 0.001,
                 0.285,
                 0.035,
                 0.016,
                 0.012],
    wt=True
)


In [6]:
# RCF conditions from Bartling et al 2021

rcf_temp = 200 + 273.15            # K 
rcf_pressure = 60 * 1e5            # Pascal
rcf_residence_time = 3             # hr
reactor_operation = 'Isothermal'
solvent_water_ratio = 9            # 9:1 methanol to water ratio on a vol basis
replacement = 1                    # once replacement of catalyst every year
biomass_deliginfication = 0.7      # 70% biomass is assumed deligniified
methanol_decomp = 0.005              # methanol decompositin rate %
cellulose_retention = 0.9          # 90% 
xylose_retention = 0.93            # 93% 
methanol_to_biomass = 90          # 90 mL / g of biomass from https://doi.org/10.1016/j.copbio.2018.12.005
hydrogen_amount = 10              # 10 L/min/dry kg biomass
catalyst_loading = 0.1            # 1:10 catalyst: dry biomass feed by wt 



lignin_monomers = 0.5
lignin_dimers = 0.25
lignin_oligomers = 0.25 





In [7]:
Methanol = bst.Chemical('Methanol')

In [10]:
Methanol.rho('l', 463.15, 6e6)

580.1765273822522

In [11]:
poplar_in = bst.MultiStream('Poplar_In',
                s=[('Poplar', 2e6)], l=[('Water', 0.2*2e6)], 
                 phases = ('s','l'), units='kg/d')

In [12]:
meoh_in = bst.Stream('Methanol_In',
                     Methanol = methanol_to_biomass*2e6*0.9, Water = methanol_to_biomass*2e6*(1/solvent_water_ratio)*0.9,
                     units = 'L/d',
                     T = 300,
                     P = 101325,
                     phase = 'l'
)

In [13]:
pump_try = bst.units.Pump(ins = meoh_in, P = 6e6)
heat_try = bst.units.HXutility(ins = pump_try-0, T = 463.15, rigorous = True)
pump_try.simulate()
heat_try.simulate()

In [14]:
heat_try.outs[0].F_vol

9979.787079720056

In [15]:
meoh_in.ivol['Methanol']/meoh_in.ivol['Water']

9.0

In [16]:
(meoh_in.F_vol*1000)/(2e6/24)

90.00000000000001

In [17]:
meoh_in.F_mass/(2e6/24)

72.5223376064465

In [18]:
solvolysis = bst.Reaction('Lignin,s -> SolubleLignin,l', reactant = 'Lignin', phases = 'sl', X = 0.7, basis = 'wt', correct_atomic_balance = False) # Since pseudocomponents, i dont think it has a compatible elemental formulae built in

In [19]:
"""
References
-----------
    [1] Bartling, Andrew W., et al. 
    "Techno-economic analysis and life cycle assessment of a biorefinery utilizing
    reductive catalytic fractionation." Energy & Environmental Science 14.8 (2021): 4147-4168.

    [2] Anderson, Eric M., et al. 
    "Flowthrough reductive catalytic fractionation of biomass." Joule 1.3 (2017): 613-622.

    [3] Froment, Gilbert F., Kenneth B. Bischoff, and Juray De Wilde. 
    Chemical reactor analysis and design. Vol. 2. New York: Wiley, 1990.
-----------
"""

import biosteam as bst, numpy as np

from typing import Optional
from biosteam.units.design_tools import (
    PressureVessel, 
)
 


__all__ = (
    'SolvolysisReactor', 'RCF-1'
    )

class SolvolysisReactor(bst.Unit, bst.units.design_tools.PressureVessel):

    """
    Plug flow reactor for solvolysis reaction, where a solvent is used to solubilize lignin present
    in plant cell wall
    



    In reality, lignin will also be partially depolymerized, but this phenomena is not captured in 
    current design but may be included in a future implementation
    Design based off [1],[2]. Pressure drop calculations from [3]

    By default,  current design supports 4 identical biomass beds, with 3 operational beds at any given time, 
    offset by 1 hour each. With a 3 hr reaction time + 1 hr turnaround, this gives near continuous throughput
    Example... t = 0 - 1 hr : bed 1, bed 2, bed 3 online, bed 4 cleaning, 
                t = 1 - 2 hr, bed 2, bed 3, bed 4 online, bed 1 cleaning
    Since at any given time, the complete throughput is mantained by 3 reactors, the total volumetric flow rate 
    of solvent is constant

    Parameters.....


    Examples


    Assumed: Extraction efficiency constant along the reaction residence time


    """



 

    auxiliary_unit_names = (
        'pump_1', 'heat_exchanger_1'
    )

    _N_ins = 2
    _N_outs = 2
    
    _units = {**PressureVessel._units,
              'Pressure drop': 'bar',
              'Diameter': 'm',
              'Length': 'm',
              'Batch time': 'hr',
              'Turnaround time': 'hr',
              'Residence time': 'hr',
              'Total volume': 'm3',
              'Reactor volume': 'm3'}
    


    # Default operating temperature [K]
    T_default: float = 463.15  # 190 C from  https://doi.org/10.1016/j.joule.2017.10.004

    #: Default operating pressure [Pa]
    P_default:  float = 6e6 # 60 MPa from https://doi.org/10.1016/j.joule.2017.10.004
    
    #: Default residence time [hr]
    tau_default: float = 3

    #: Default cleaning and unloading time (hr).
    tau_0_default: float  = 1 # from https://doi.org/10.1039/D1EE01642C
    
    # Default superficial velocity of solvent (m/s)
    superficial_velocity_default: float = 1 # Just assumed 

    # Default methanol decomposition (%)
    methanol_decomposition_default: float = 0.005 # from https://doi.org/10.1039/D1EE01642C

    # Default poplar bed void fraction (epsilon)
    void_frac_default: float = 0.3    


    # Default working volume fraction 
    working_vol_default: float = 0.8
 
    # Default poplar diameter [m]
    poplar_diameter_default: float = 0.004   
    



    def _init(
            self,
            T: Optional[float] = None, 
            P: Optional[float] = None,
            tau: Optional[float] = None,
            vessel_material: Optional[str] = None,
            vessel_type: Optional[str] = None,
            tau_0: Optional[float] = None,  
            superficial_velocity: Optional[float] = None,
            methanol_decomposition: Optional[float] = None,
            void_frac: Optional[float] = None,
            working_vol: Optional[float] = None,
            poplar_diameter: Optional[float] = None,
            N_beds = 16, *,
            reaction
            ):
        
        
        self.T = self.T_default if T is None else T
        self.P = self.P_default if P is None else P
        self.tau = self.tau_default if tau is None else tau
        self.vessel_material = 'Stainless steel 316' if vessel_material is None else vessel_material
        self.vessel_type = 'Vertical' if vessel_type is None else vessel_type
        self.tau_0 = self.tau_0_default if tau_0 is None else tau_0
        self.superficial_velocity = self.superficial_velocity_default if superficial_velocity is None else superficial_velocity
        self.methanol_decomposition = self.methanol_decomposition_default if methanol_decomposition is None else methanol_decomposition
        self.void_frac = self.void_frac_default if void_frac is None else void_frac
        self.working_vol = self.working_vol_default if working_vol is None else working_vol
        self.poplar_diameter = self.poplar_diameter_default if poplar_diameter is None else poplar_diameter
        if N_beds != 16:
            raise ValueError('only 4 biomass beds are valid')
        self.reaction = reaction
        self.N_beds = N_beds
        pump_1 = self.auxiliary('pump_1', bst.Pump, ins = self.ins[1])
        heat_exchanger_1 = self.auxiliary('heat_exchanger_1', bst.HXutility, ins = pump_1.outs[0])


    def _size_bed(self):

        #### Reactor volume sizing ########

        cycle_time = self.tau + self.tau_0    # Total time for a cycle of operation (includes reaction + cleaning-loading-unloading)
        occupied_frac = 1 - self.void_frac    # ratio of occupied biomass fraction/total biomass bed
        working_vol = self.working_vol        # Working volume fraction for a reactor to allow suficient mass transfer by allowing enough contact time 
        solvent = self.ins[1]
        liquid_V_w = solvent.F_vol * cycle_time   # [m3] Total liquid working volume
        liquid_V_w_pr = liquid_V_w/self.N_beds    # [m3] Liquid working volume per reactor 
        V_actual = liquid_V_w_pr/(occupied_frac * working_vol)  # [m3]
        total_volume = V_actual * self.N_beds

        ##### Reactor diameter and length ########
        u  =self.superficial_velocity

        N_operating = self.N_beds *(self.tau/cycle_time) # Number of beds operating at any given time
        Q_per_reactor_m3 = solvent.F_vol/N_operating     # [m3/hr] Volumetric flow rate processed by any reactor
        self.area = A  = Q_per_reactor_m3/u
        self.diameter = diameter = 2 * (A/np.pi) ** 0.5
        self.length = length = V_actual/A

        
        return length, diameter, V_actual, total_volume        

        
    def _run(self):
        biomass, solvent = self.ins
        used_biomass, used_solvent = self.outs
        used_solvent.copy_like(solvent) 
        used_solvent.imass['Methanol'] = used_solvent.imass['Methanol']*(1-self.methanol_decomposition) # 0.5% methanol lost due to decomposition
        used_biomass.copy_like(biomass)
        used_solvent.P = self.P
        used_solvent.T = self.T

        self.reaction(used_biomass)
        

        solubilized_lignin = used_biomass.imass['SolubleLignin'] 
        extractives = used_biomass.imass['Extract'] # From Table S1 https://www.rsc.org/suppdata/d1/gc/d1gc01591e/d1gc01591e1.pdf,
                                                        # it follows that the extractives component of poplar is 'extracted' in the solvent stream
        used_solvent.imass['Extract'] += extractives
        used_biomass.imass['s', 'Extract'] = 0 

        acetate = used_biomass.imass['Acetate']
        used_solvent.imass['Acetate'] = acetate
        used_biomass.imass['s', 'Acetate'] = 0

        arabinan_retained = used_biomass.imass['Arabinan']*0.5
        used_biomass.imass['s', 'Arabinan'] = arabinan_retained
        used_solvent.imass['Arabinan'] = arabinan_retained

        galactan_retained = used_biomass.imass['Galactan']*0.5
        used_biomass.imass['s', 'Galactan'] = galactan_retained
        used_solvent.imass['Galactan'] = galactan_retained

        
        used_solvent.imass['SolubleLignin'] += solubilized_lignin # Soluble lignin part of solvent stream 
        used_biomass.imass['l', 'SolubleLignin'] = 0 # No soluble lignin in biomass





    def _calculate_pressure_drop(self):

        D = self.poplar_diameter # [m] poplar particle diameter
        rho_solv = self.ins[1].rho  # [kg/m3] methanol water density
        mu = self.ins[1].get_property('mu', 'kg/m/s') # [Pa s] methanol water viscosity 
        epsilon = self.void_frac # Void fraction 
        u = self.superficial_velocity # [m/s] superficial velocity          
        
        Re = (D*rho_solv*u)/mu
        if Re/(1-epsilon) < 500: # Erun equation
            dP = ((1-epsilon)/(epsilon**3))*(1.75+(150*(1-epsilon)/Re))
        elif 1000 < Re/(1-epsilon) < 5000: # Handley and Heggs
            dP = ((1-epsilon)/(epsilon**3))*(1.24+(368*(1-epsilon)/Re))
        else: # Hicks equation which fits in Wentz and Thodos results for very high Re
            dP = 6.8*(((1-epsilon)**1.2)/epsilon**3)*Re**-0.2
        return dP

        

    def _design(self):
        diameter, length, V_actual, total_volume = self._size_bed()   # Calling size bed function to determine diameter and length 


        cycle_time = self.tau + self.tau_0

        self.set_design_result('Diameter', 'm', diameter)  
        self.set_design_result('Length', 'm', length)
        self.set_design_result('Reactor volume', 'm3', V_actual)
        self.set_design_result('Total volume', 'm3', total_volume)
        self.set_design_result('Residence time', 'hr', self.tau)
        self.set_design_result('Turnaround time', 'hr', self.tau_0)
        self.set_design_result('Batch time', 'hr', cycle_time)


        
        # Calculates weight based off pressure, diameter and length
        # Adds vcessel type wall thickness, vessel weight, diameter and length to dictionary
        # But diameter and length are already there because of set_design_result above
        self.design_results.update(
            self._vertical_vessel_design(    
                self.P*(1/6894.76),
                self.design_results['Diameter']*3.28084,
                self.design_results['Length']*3.28084
            )
        )
        
                            

        pressure_drop = self._calculate_pressure_drop()                  
        
        self.set_design_result('Pressure drop', 'bar', pressure_drop)
        self.pump_1.P = (self.P - self.ins[1].P) + (pressure_drop*1e5)
        self.pump_1.simulate()

        self.heat_exchanger_1.T = self.T
        self.heat_exchanger_1.simulate()
        





        

    def _cost(self):
        design = self.design_results # Calling the dictionary used to store design results in design method above 

        baseline_purchase_costs = self.baseline_purchase_costs # Dictionary for storing baseline costs

        weight = design['Weight']  # weight parameter stores the value from the 'Weight' key in the design dictionnary

        # Calculates the baseline purchase cost based off diameter length and weight
        baseline_purchase_costs.update( 
            self._vessel_purchase_cost(weight, design['Diameter'], design['Length'])
        )

        self.parallel['self'] = self.N_beds # Used to create multiple of the same beds
        # self.parallel['pump_1'] = 1


        
        """

        ---------

        - Duty requirements are too high - need to check thermodynamics of the solvolysis step
        and modify requirements if needed
          
        - Might need to alter data on working volume, void fraction of poplar, poplar particle size,
        for more realistic modeling

        ----------

        """



        

        


In [20]:
solvolysis_reactor = SolvolysisReactor(ins = (poplar_in, meoh_in), N_beds = 16, void_frac= 0.4, superficial_velocity = 1, reaction = solvolysis)
solvolysis_reactor.simulate()

c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\IPython\core\interactiveshell.py:3579: RuntimeWarning: undocked inlet Methanol_In from P1; Methanol_In is now docked at R1
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\hwadg\AppData\Local\Temp\ipykernel_35204\505073771.py:254: CostWarning: <SolvolysisReactor: R1> Vertical vessel weight (2.918e+06 lb) is out of bounds (4200 to 1e+06 lb) for cost correlation
  self._vertical_vessel_design(
C:\Users\hwadg\AppData\Local\Temp\ipykernel_35204\505073771.py:254: CostWarning: <SolvolysisReactor: R1> Vertical vessel length (92.55 ft) is out of bounds (12 to 40 ft) for cost correlation
  self._vertical_vessel_design(
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\units\_pump.py:224: RuntimeWarning: <Pump: pump_1> no pump type available at current power (2.37e+04 hp), head (3.52e+03 ft), kinematic viscosity (7.38e-07 m2/s), and NPSH (35.4 ft); assuming centrigugal pump
  warn(f'{repr(self)} no pump type availab

In [21]:
((solvolysis_reactor.outs[1].imass['Extract'] + solvolysis_reactor.outs[1].imass['Acetate']
  + solvolysis_reactor.outs[1].imass['Arabinan'] + solvolysis_reactor.outs[1].imass['Galactan'])*24*330)/1e6

38.93999999999999

In [ ]:
(solvolysis_reactor.outs[1].imass['Extract']*24*330)/1e6

10.560000000000002

In [22]:
solvolysis_reactor.results()

Solvolysis reactor                                         Units        R1
Electricity         Power                                     kW  2.83e+05
                    Cost                                  USD/hr  2.21e+04
High pressure steam Duty                                   kJ/hr  6.48e+10
                    Flow                                 kmol/hr  2.01e+06
                    Cost                                  USD/hr  6.38e+05
Design              Diameter                                   m      20.5
                    Length                                     m      92.6
                    Reactor volume                            m3  3.91e+03
                    Total volume                              m3  6.25e+04
                    Residence time                            hr         3
                    Turnaround time                           hr         1
                    Batch time                                hr         4
                    Vessel type                                   Vertical
                    Weight                                    lb  2.92e+06
                    Wall thickness                            in      9.63
                    Pressure drop                            bar      10.3
Purchase cost       Vertical pressure vessel (x16)           USD  9.95e+07
                    Platform and ladders (x16)               USD  1.44e+06
                    Pump 1 - Pump (x16)                      USD  1.18e+08
                    Pump 1 - Motor (x768)                    USD   3.2e+04
                    Heat exchanger 1 - Floating head...      USD  6.49e+07
Total purchase cost                                          USD  2.84e+08
Utility cost                                              USD/hr  6.61e+05

In [ ]:
solvolysis_reactor-1

Stream: s89 from <SolvolysisReactor: R42> to <Mixer: M4>
phase: 'l', T: 463.15 K, P: 6e+06 Pa
flow (kmol/hr): Water          4.15e+04
                Extract        7.4
                Acetate        48.6
                SolubleLignin  109
                Arabinan       0.631
                Galactan       3.6
                Methanol       1.64e+05


In [23]:
hydrogen_in = bst.Stream('Hydrogen_In', 
                         Hydrogen = meoh_in.F_vol*100,   # Anderson et al, had MeOH/H2 feeds as 1:100 vol:vol ratio
                         units = 'm3/hr',
                         T = 190+273.15,
                         P = 6e6,
                         phase = 'g'
)




In [ ]:
solvolysis_reactor-1

Stream: s89 from <SolvolysisReactor: R42> to <Mixer: M4>
phase: 'l', T: 463.15 K, P: 6e+06 Pa
flow (kmol/hr): Water          4.15e+04
                Extract        7.4
                Acetate        48.6
                SolubleLignin  109
                Arabinan       0.631
                Galactan       3.6
                Methanol       1.64e+05


In [24]:
heaterz = bst.units.HXutility(ins = solvolysis_reactor-1, T = 464.15, rigorous = True)
heaterz.simulate()

In [25]:
mixer = bst.units.Mixer(ins = (hydrogen_in, solvolysis_reactor-1), rigorous = True)
mixer.simulate()

c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\IPython\core\interactiveshell.py:3579: RuntimeWarning: undocked inlet s4 from H2; s4 is now docked at M1
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\thermosteam\equilibrium\bubble_point.py:128: RuntimeWarning: Hydrogen has no defined Dortmund groups; functional group interactions are ignored
  self.gamma = thermo.Gamma(chemicals)


In [26]:
hx_1 = bst.units.HXutility(ins = mixer-0, T = 463.15, rigorous = True)
hx_1.simulate()

In [27]:
hx_1-0

MultiStream: s7 from <HXutility: H3>
phases: ('g', 'l'), T: 463.15 K, P: 6e+06 Pa
flow (kmol/hr): (g) Water          4.15e+04
                    Acetate        48.6
                    Methanol       1.64e+05
                    Hydrogen       1.13e+06
                (l) Extract        7.4
                    SolubleLignin  109
                    Arabinan       0.631
                    Galactan       3.6


In [28]:
solvolysis_reactor.outs[1].show(flow = 'kg/hr', composition = True)

Stream: s4 from <SolvolysisReactor: R1> to <Mixer: M1>
phase: 'l', T: 463.15 K, P: 6e+06 Pa
composition (%): Water          12.4
                 Extract        0.0221
                 Acetate        0.0483
                 SolubleLignin  0.275
                 Arabinan       0.00138
                 Galactan       0.00966
                 Methanol       87.3
                 -------------  6.04e+06 kg/hr


In [29]:
hydrogenolysis = bst.ParallelReaction([
    bst.Reaction('SolubleLignin,l ->Propylguaiacol,l', reactant = 'SolubleLignin', X = 0.25),
    bst.Reaction('SolubleLignin,l ->Propylsyringol,l', reactant = 'SolubleLignin', X = 0.25),
    bst.Reaction('SolubleLignin,l -> Syringaresinol,l', reactant= 'SolubleLignin', X = 0.125),
    bst.Reaction('SolubleLignin,l -> G_Dimer,l', reactant= 'SolubleLignin', X = 0.125),
    bst.Reaction('SolubleLignin,l -> S_Oligomer,l', reactant= 'SolubleLignin', X = 0.125),
    bst.Reaction('SolubleLignin,l -> G_Oligomer,l', reactant= 'SolubleLignin', X = 0.125),
])

In [30]:
hydrogenolysis

ParallelReaction (by mol):
index  stoichiometry                        reactant           X[%]
[0]    SolubleLignin,l -> Propylguaiacol,l  SolubleLignin,l   25.00
[1]    SolubleLignin,l -> Propylsyringol,l  SolubleLignin,l   25.00
[2]    SolubleLignin,l -> Syringaresinol,l  SolubleLignin,l   12.50
[3]    SolubleLignin,l -> G_Dimer,l         SolubleLignin,l   12.50
[4]    SolubleLignin,l -> S_Oligomer,l      SolubleLignin,l   12.50
[5]    SolubleLignin,l -> G_Oligomer,l      SolubleLignin,l   12.50


In [31]:
import biosteam as bst, numpy as np

from typing import Optional
from biosteam.units.design_tools import (
    PressureVessel, 
)
 


__all__ = (
    'HydrogenolysisReactor', 'RCF-2'
    )

class HydrogenolysisReactor(bst.Unit, bst.units.design_tools.PressureVessel):


    auxiliary_unit_names = (
        'heat_exchanger_2'
    )

    _N_ins = 2
    _N_outs = 1
    
    _units = {**PressureVessel._units,
              # 'Pressure drop': 'bar',
              'Diameter': 'm',
              'Length': 'm',
              'Residence time': 'hr',
              'Reactor volume': 'm3'}
    


    # Default operating temperature [K]
    T_default: float = 463.15  # 190 C from  https://doi.org/10.1016/j.joule.2017.10.004

    #: Default operating pressure [Pa]
    P_default:  float = 6e6 # 60 MPa from https://doi.org/10.1016/j.joule.2017.10.004
    
    #: Default residence time [hr]
    tau_default: float = 3

    # Default superficial velocity of solvent (m/s)
    superficial_velocity_default: float = 1 # Just assumed 

    # Default catalyst bed void fraction (epsilon)
    void_frac_default: float = 0.3    


    # Default working volume fraction 
    working_vol_default: float = 0.8
 
    # Default catayyst diameter [m]
    # poplar_diameter_default: float = 0.004   
    



    def _init(
            self,
            T: Optional[float] = None, 
            P: Optional[float] = None,
            tau: Optional[float] = None,
            vessel_material: Optional[str] = None,
            vessel_type: Optional[str] = None,
            superficial_velocity: Optional[float] = None,
            void_frac: Optional[float] = None,
            working_vol: Optional[float] = None,
            *,
            reaction
            ):
        
        
        self.T = self.T_default if T is None else T
        self.P = self.P_default if P is None else P
        self.tau = self.tau_default if tau is None else tau
        self.vessel_material = 'Stainless steel 316' if vessel_material is None else vessel_material
        self.vessel_type = 'Vertical' if vessel_type is None else vessel_type
        self.superficial_velocity = self.superficial_velocity_default if superficial_velocity is None else superficial_velocity
        self.void_frac = self.void_frac_default if void_frac is None else void_frac
        self.working_vol = self.working_vol_default if working_vol is None else working_vol
        self.reaction = reaction
        #pump_1 = self.auxiliary('pump_1', bst.Pump, ins = self.ins[1])
        heat_exchanger_2 = self.auxiliary('heat_exchanger_2', bst.HXutility, ins = mixer-0)


    def _size_bed(self):

        #### Reactor volume sizing ########

        residence_time = self.tau
        occupied_frac = 1 - self.void_frac  
        working_vol = self.working_vol        
        influent = self.ins[0]
        liquid_V_w = influent.F_vol * residence_time # [m3] Total liquid working volume
        V_actual = liquid_V_w/(occupied_frac * working_vol)  # [m3]

        ##### Reactor diameter and length ########
        u  = self.superficial_velocity

        Q_per_reactor_m3 = solvent.F_vol    # [m3/hr] Volumetric flow rate processed by any reactor
        self.area = A  = Q_per_reactor_m3/u
        self.diameter = diameter = 2 * (A/np.pi) ** 0.5
        self.length = length = V_actual/A

        
        return length, diameter, V_actual
        
    def _run(self):
        solvent, hydrogen = self.ins
        effluent,= self.outs

        effluent.copy_like(solvent)


        h2 = hydrogen.imass['Hydrogen']

        effluent.imass['Hydrogen'] = h2

        effluent.T = self.T # Assuming isothermal operation
        effluent.P = self.P # Assuming no P drop





        self.reaction(effluent)
        




    #def _calculate_pressure_drop(self):
        # NOT OPERATIONAL FOR HYDROGENOLYSIS REACTOR

        #D = self.poplar_diameter # [m] poplar particle diameter
        #rho_solv = self.ins[1].rho  # [kg/m3] methanol water density
        #mu = self.ins[1].get_property('mu', 'kg/m/s') # [Pa s] methanol water viscosity 
        #epsilon = self.void_frac # Void fraction 
        #u = self.superficial_velocity # [m/s] superficial velocity          
        
        #Re = (D*rho_solv*u)/mu
        #if Re/(1-epsilon) < 500: # Erun equation
        #    dP = ((1-epsilon)/(epsilon**3))*(1.75+(150*(1-epsilon)/Re))
        #elif 1000 < Re/(1-epsilon) < 5000: # Handley and Heggs
        #    dP = ((1-epsilon)/(epsilon**3))*(1.24+(368*(1-epsilon)/Re))
        #else: # Hicks equation which fits in Wentz and Thodos results for very high Re
        #    dP = 6.8*(((1-epsilon)**1.2)/epsilon**3)*Re**-0.2
        #return dP

        

    def _design(self):
        diameter, length, V_actual = self._size_bed()   # Calling size bed function to determine diameter and length 


        cycle_time = self.tau + self.tau_0

        self.set_design_result('Diameter', 'm', diameter)  
        self.set_design_result('Length', 'm', length)
        self.set_design_result('Reactor volume', 'm3', V_actual)
        self.set_design_result('Residence time', 'hr', self.tau)


        
        # Calculates weight based off pressure, diameter and length
        # Adds vcessel type wall thickness, vessel weight, diameter and length to dictionary
        # But diameter and length are already there because of set_design_result above
        self.design_results.update(
            self._vertical_vessel_design(    
                self.P*(1/6894.76),
                self.design_results['Diameter']*3.28084,
                self.design_results['Length']*3.28084
            )
        )
        
                            

        #pressure_drop = self._calculate_pressure_drop()                  
        
        #self.set_design_result('Pressure drop', 'bar', pressure_drop)
        #self.pump_1.P = (self.P - self.ins[1].P) + (pressure_drop*1e5)
        #self.pump_1.simulate()

        self.heat_exchanger_2.T = self.T
        self.heat_exchanger_2.simulate()
        





        

    def _cost(self):
        design = self.design_results # Calling the dictionary used to store design results in design method above 

        baseline_purchase_costs = self.baseline_purchase_costs # Dictionary for storing baseline costs

        weight = design['Weight']  # weight parameter stores the value from the 'Weight' key in the design dictionnary

        # Calculates the baseline purchase cost based off diameter length and weight
        baseline_purchase_costs.update( 
            self._vessel_purchase_cost(weight, design['Diameter'], design['Length'])
        )



        
        """

        ---------

        - Duty requirements are too high - need to check thermodynamics of the solvolysis step
        and modify requirements if needed
          
        - Might need to alter data on working volume, void fraction of poplar, poplar particle size,
        for more realistic modeling

        ----------

        """



        

        


In [ ]:
mixer-0

MultiStream: s92 from <Mixer: M4> to <HXutility: heat_exchanger_2>
phases: ('g', 'l'), T: 393.99 K, P: 6e+06 Pa
flow (kmol/hr): (g) Water          1.69e+04
                    Acetate        7.44
                    Methanol       1.02e+05
                    Hydrogen       1.13e+06
                (l) Water          2.46e+04
                    Extract        7.4
                    Acetate        41.1
                    SolubleLignin  109
                    Arabinan       0.631
                    Galactan       3.6
                    Methanol       6.29e+04


In [ ]:
solvolysis_reactor-1

Stream: s89 from <SolvolysisReactor: R42> to <HydrogenolysisReactor: R45>
phase: 'l', T: 463.15 K, P: 6e+06 Pa
flow (kmol/hr): Water          4.15e+04
                Extract        7.4
                Acetate        48.6
                SolubleLignin  109
                Arabinan       0.631
                Galactan       3.6
                Methanol       1.64e+05


In [32]:
reactor = HydrogenolysisReactor(ins = (solvolysis_reactor-1, hydrogen_in), reaction = hydrogenolysis)
reactor.simulate()

c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\IPython\core\interactiveshell.py:3579: RuntimeWarning: undocked inlet s4 from M1; s4 is now docked at R2
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\IPython\core\interactiveshell.py:3579: RuntimeWarning: undocked inlet Hydrogen_In from M1; Hydrogen_In is now docked at R2
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\hwadg\AppData\Local\Temp\ipykernel_35204\446644315.py:83: RuntimeWarning: undocked inlet s6 from H3; s6 is now docked at heat_exchanger_2
  heat_exchanger_2 = self.auxiliary('heat_exchanger_2', bst.HXutility, ins = mixer-0)


IndexError: too many indices for array; array is 1-dimensional, but 2 were indexed

In [ ]:
reactor

RCFReactor: R2
ins...
[0] Poplar_In  
    phases: ('l', 's'), T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): (l) Water      925
                    (s) Sucrose    0.243
                        Extract    7.4
                        Acetate    48.6
                        Ash        1e+03
                        Lignin     156
                        Cellulose  238
                        Xylan      84.5
                        ...        27.5
[1] Methanol_In  
    phase: 'l', T: 300 K, P: 100000 Pa
    flow (kmol/hr): Water     5.53e+03
                    Methanol  2.2e+04
outs...
[0] s3  
    phases: ('l', 's'), T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): (l) Water      925
                    (s) Sucrose    0.243
                        Extract    7.4
                        Acetate    48.6
                        Ash        1e+03
                        Lignin     46.8
                        Cellulose  238
                        Xylan      84.5
                        ...       

In [ ]:
reactor.results()

RCFReactor                                                 Units        R2
Electricity         Power                                     kW  8.05e+03
                    Cost                                  USD/hr       630
High pressure steam Duty                                   kJ/hr  2.16e+09
                    Flow                                 kmol/hr  6.71e+04
                    Cost                                  USD/hr  2.13e+04
Design              Diameter                                  ft      12.9
                    Length                                    ft      75.6
                    Reactor volume                            m3  2.47e+03
                    Vessel type                                   Vertical
                    Weight                                    lb   8.7e+05
                    Wall thickness                            in       6.1
                    Pressure drop                            bar      6.74
Purchase cost       Vertical pressure vessel (x4)            USD  9.02e+06
                    Platform and ladders (x4)                USD  2.21e+05
                    Pump 1 - Pump (x4)                       USD  2.53e+06
                    Pump 1 - Motor (x24)                     USD  1.08e+06
                    Heat exchanger 1 - Floating head...      USD  1.46e+06
Total purchase cost                                          USD  1.43e+07
Utility cost                                              USD/hr  2.19e+04